# Project 2 - Fieldline and particle tracing

In this project we take a look at tracing field lines and particles in the magnetic field generated by the coils.

To get started, we'll have a look at how to evaluate the magnetic field at a given point. We begin by setting up our toy problem again.

In [ ]:
from pyplasmaopt import *
nfp = 2
(coils, currents, expansion_axis, eta_bar) = get_24_coil_data(nfp=nfp, ppp=10, at_optimum=True)
stellarator = CoilCollection(coils, currents, nfp, True)

Now let's write a little helper function, that evaluates the Biot Savart law

In [ ]:
import cppplasmaopt as cpp
coil_points = [coil.gamma for coil in stellarator.coils]
coil_tangents = [coil.dgamma_by_dphi[:, 0, :] for coil in stellarator.coils]
def biot_savart(stellarator, xyz):
    xyz = xyz.reshape((1, 3))
    Bxyz = np.zeros((1, 3))
    cpp.biot_savart_B_only(xyz, coil_points, coil_tangents, stellarator.currents, Bxyz)
    return Bxyz

Let's check that this makes sense. At the point (1, 0, 0), we expect the field to mostly point in $y$ direction.

In [ ]:
print(biot_savart(stellarator, np.asarray([1., 0., 0.])))

## Tasks

1) Given an initial point $X_0$, the magnetic fieldline starting at $X_0$ can be obtained by solving the following initial value problem

$$ X(0) = X_0, \quad \dot X(t) = B(X(t)).$$

At the optimum, the expansion axis should align with the true magnetic axis of the field. The goal is now to check this by picking a point on the expansion axis, `expansion_axis.gamma[0,:]`, and solving the above ODE starting there. Does the fieldline bite its tail? Hint: You can add extra data to the `plot_stellarator` function, via `plot_stellarator(stellarator, axis=expansion_axis, extra_data=[ys])` where `ys` is a numpy array of shape `(n, 3)` for some `n`. You can also pass a list of more than one array to plot multiple trajectories.

2) Given an initial point $X_0$ and initial velocity $V_0$, the motion of a particle with mass $m$ and charge $q$ is given by

$$ X(0) = X_0,\quad \dot X(0) = V_0,\quad \ddot X(t) = \frac qm \dot X(t) \times B.$$

Take a particle with mass $m = 6.644657230*10^{-27}$ and charge $q = 2*1.602176634*10^{-19}$ and start it at $X_0 =$ `expansion_axis.gamma[0,:]` and $V_0 = [10^5, 2*10^5, 0]$ and track the particle up to $T_\max = 10^{-4}$. Plot its trajectory. Create 2D plots of t->x(t), t->y(t), t->z(t) and zoom in closely. What do you observe? Now compare with a particle started at the same point, but with $V_0 = [3*10^5, 2*10^5, 0]$. How do the trajectories differ?

## Solution

### Task 1

In [ ]:
from scipy.integrate import solve_ivp, RK45, OdeSolution
def rhs(t, xyz):
    return biot_savart(stellarator, xyz)
y0 = expansion_axis.gamma[0, :]
solver = RK45(rhs, 0, y0, 10, rtol=1e-9, atol=1e-09)
ys = []
ts = []
while solver.t < 10:
    solver.step()
    ys.append(solver.y)
    ts.append(solver.t)
ys = np.asarray(ys)
plot_stellarator(stellarator, axis=expansion_axis, extra_data=[ys])

### Task 2

We rewrite the 2nd order ODE as a system of two ODES:

$$ \begin{bmatrix}\dot X \\ \dot V \end{bmatrix} = \begin{bmatrix} V(t) \\ \frac qm \dot V(t) \times B(X(t)) \end{bmatrix}  $$

In [ ]:
from scipy.integrate import solve_ivp, RK45, OdeSolution
m = 6.644657230*1e-27
q = 2*1.602176634*1e-19
def rhs_part(t, y):
    X = y[0:3]
    V = y[3:6]
    B = biot_savart(stellarator, X)
    rhs = np.zeros_like(y)
    rhs[0:3] = V
    rhs[3:6] = (q/m) * np.cross(V, B)
    return rhs



In [ ]:
v0 = np.asarray([1e5, 2e5, 0])
y0 = np.zeros((6, ))
y0[0:3] = expansion_axis.gamma[0, :]
y0[3:6] = v0
tmax = 1e-4
solver = RK45(rhs_part, 0, y0, tmax, rtol=1e-5, atol=1e-5)
ys = []
ts = []
while solver.t < tmax:
    solver.step()
    ys.append(solver.y)
    ts.append(solver.t)
ysa = np.asarray(ys)

In [ ]:
plot_stellarator(stellarator, axis=expansion_axis, extra_data=[ysa[:, 0:3]])

In [ ]:
import plotly.express as px
fig = px.line(ysa[:, 0:3])
fig.show()

In [ ]:
v0 = np.asarray([3e5, 2e5, 0])
y0 = np.zeros((6, ))
y0[0:3] = expansion_axis.gamma[0, :]
y0[3:6] = v0
tmax = 1e-4
solver = RK45(rhs_part, 0, y0, tmax, rtol=1e-5, atol=1e-5)
ys = []
ts = []
while solver.t < tmax:
    solver.step()
    ys.append(solver.y)
    ts.append(solver.t)
ysb = np.asarray(ys)

In [ ]:
import plotly.express as px
fig = px.line(ysb[:, 0:3])
fig.show()

In [ ]:
plot_stellarator(stellarator, axis=expansion_axis, extra_data=[ysa[:, 0:3], ysb[:, 0:3]])

### Observation: 
The second particle has an initial velocity with much larger component in perpendicular direction to the initial magnetic field. Consequently, its gyro radius is larger. We also observe that it deviates from the magnetic axis quicker than the first particle.